<a href="https://colab.research.google.com/github/Marcin19721205/Wprowadzenie-Do-Sieci-Neuronowych/blob/main/11_Keras03_overfitting_underfitting01MJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Główne problemy uczenia maszynowego: przeuczenie (overfitting) oraz niedouczenie (underfitting)

#Klasyfikacja Binarna - overfitting & underfitting

#Praca z danymi tekstowymi

>Celem tego notebook'a jest pokazanie przykładów zbyt dobrego dopasowanie modelu do danych uczących (przeuczenie) oraz zbyt słabego dopasowania modelu do danych uczących (niedouczenie).
>
>Wykorzystamy zbiór z bilioteki Keras składający się z 50000 recenzji filmów oznaczonych sentymentem: pozytywny/negatywny. Recenzje są wstępnie przetworzone, a każda recenzja jest zakodowana jako sekwencja indeksów słów. Słowa są indeksowane według ogólnej częstotliwości w zbiorze danych. Na przykład liczba 5 oznacza piąte najczęściej pojawiające się słowo w danych. Liczba 0 nie oznacza określonego słowa.

### Spis treści
1. [Import bibliotek](#a1)
2. [Załadowanie i przygotowanie danych](#a2)
3. [Budowa modelu bazowego](#a3)
4. [Budowa 'mniejszego' modelu](#a4)    
5. [Budowa 'większego' modelu](#a5)
6. [Porównanie wydajności modeli](#a6)
7. [Metody regularyzacji](#a7)

### <a name='a1'></a> 1. Import bibliotek

In [2]:
%tensorflow_version 2.x
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow.keras.datasets import imdb #zbiór danych z którego korzystamy
from tensorflow.keras.datasets.imdb import get_word_index #mapuje wartrośc liczbową na słowo
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sns.set()
tf.__version__

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


'2.19.0'

### <a name='a2'></a> 2. Załadowanie i przygotowanie danych

In [3]:
NUM_WORDS = 10000   # 10000 najczęściej pojawiających się słów
INDEX_FROM = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)

In [4]:
print(f'train_data shape: {train_data.shape}')
print(f'test_data shape: {test_data.shape}')

train_data shape: (25000,)
test_data shape: (25000,)


In [5]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


Przekształca powyższą listę numeryczną na słowa

np <UNK> wypadło poza 10000 slów rozpoznawanych  

In [6]:
word_to_idx = get_word_index()
word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
word_to_idx['<PAD>'] = 0
word_to_idx['<START>'] = 1
word_to_idx['<UNK>'] = 2
word_to_idx['<UNUSED>'] = 3
idx_to_word = {v: k for k, v in word_to_idx.items()}
list(idx_to_word.items())[:10]
print(' '.join(idx_to_word[idx] for idx in train_data[0]))


<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

labele jako klasyfikacja binarna

In [7]:
train_labels[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [8]:
def multi_hot_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0
    return results

train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
train_data.shape

(25000, 10000)

In [9]:
test_data.shape

(25000, 10000)

### <a name='a3'></a> 3. Budowa modelu bazowego

# 16 neuronnów w ukrtytch

In [14]:
baseline_model = Sequential()
baseline_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
baseline_model.add(Dense(16, activation='relu'))
baseline_model.add(Dense(1, activation='sigmoid'))

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

baseline_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 16)             │       160,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,305 (626.19 KB)

 Trainable params: 160,305 (626.19 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
#dodany kod, bo się wymuiary nie zgadzały
# etykiety 0/1 jako kolumna, float32
train_labels = np.asarray(train_labels, dtype='float32').reshape(-1, 1)
test_labels  = np.asarray(test_labels,  dtype='float32').reshape(-1, 1)

baseline_history = baseline_model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512,
    validation_data=(test_data, test_labels)
)

##################################################################
#                   KURS
##################################################################

baseline_history = baseline_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 110ms/step - accuracy: 1.0000 - loss: 3.3513e-04 - val_accuracy: 0.8467 - val_loss: 1.4417
Epoch 2/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 9s 92ms/step - accuracy: 1.0000 - loss: 3.2196e-04 - val_accuracy: 0.8468 - val_loss: 1.4554
Epoch 3/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 1.0000 - loss: 3.0904e-04 - val_accuracy: 0.8471 - val_loss: 1.4696
Epoch 4/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 1.0000 - loss: 2.8859e-04 - val_accuracy: 0.8472 - val_loss: 1.4825
Epoch 5/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 1.0000 - loss: 2.6516e-04 - val_accuracy: 0.8468 - val_loss: 1.4953
Epoch 6/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 1.0000 - loss: 2.4130e-04 - val_accuracy: 0.8465 - val_loss: 1.5079
Epoch 7/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 1.0000 - loss: 2.4182e-04 - val_accuracy: 0.8470 - val_loss: 1.5211
Epoch 8/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 1.0000 - loss: 2.22

In [28]:
train_labels = train_labels.reshape(-1, 1)
test_labels = test_labels.reshape(-1, 1)

print(f'train_labels shape: {train_labels.shape}')
print(f'test_labels shape: {test_labels.shape}')

train_labels shape: (25000, 1)
test_labels shape: (25000, 1)


### <a name='a4'></a> 3. Budowa 'mniejszego' modelu

# 4 neurony w ukrytych

In [29]:
smaller_model = Sequential()
smaller_model.add(Dense(4, activation='relu', input_shape=(NUM_WORDS,)))
smaller_model.add(Dense(4, activation='relu'))
smaller_model.add(Dense(1, activation='sigmoid'))

smaller_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

smaller_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 4)              │        40,004 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 4)              │            20 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,029 (156.36 KB)

 Trainable params: 40,029 (156.36 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
smaller_history = smaller_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 90ms/step - accuracy: 0.6254 - binary_crossentropy: 0.6745 - loss: 0.6745 - val_accuracy: 0.8049 - val_binary_crossentropy: 0.5957 - val_loss: 0.5957
Epoch 2/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8171 - binary_crossentropy: 0.5561 - loss: 0.5561 - val_accuracy: 0.8474 - val_binary_crossentropy: 0.5021 - val_loss: 0.5021
Epoch 3/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.8874 - binary_crossentropy: 0.4476 - loss: 0.4476 - val_accuracy: 0.8813 - val_binary_crossentropy: 0.4009 - val_loss: 0.4009
Epoch 4/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step - accuracy: 0.9148 - binary_crossentropy: 0.3277 - loss: 0.3277 - val_accuracy: 0.8872 - val_binary_crossentropy: 0.3156 - val_loss: 0.3156
Epoch 5/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.9289 - binary_crossentropy: 0.2379 - loss: 0.2379 - val_accuracy: 0.8887 - val_binary_crossentropy: 0.2906 - val_loss: 0.2906
Epoch 6/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms

### <a name='a5'></a> 4. Budowa 'większego' modelu

# 512 neuronów w ukrytych

In [31]:
bigger_model = Sequential()
bigger_model.add(Dense(512, activation='relu', input_shape=(NUM_WORDS,)))
bigger_model.add(Dense(512, activation='relu'))
bigger_model.add(Dense(1, activation='sigmoid'))

bigger_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

bigger_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 512)            │     5,120,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,383,681 (20.54 MB)

 Trainable params: 5,383,681 (20.54 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
bigger_history = bigger_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 22s 430ms/step - accuracy: 0.7738 - binary_crossentropy: 0.4619 - loss: 0.4619 - val_accuracy: 0.8776 - val_binary_crossentropy: 0.2957 - val_loss: 0.2957
Epoch 2/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 18s 373ms/step - accuracy: 0.9475 - binary_crossentropy: 0.1475 - loss: 0.1475 - val_accuracy: 0.8719 - val_binary_crossentropy: 0.3361 - val_loss: 0.3361
Epoch 3/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 19s 394ms/step - accuracy: 0.9854 - binary_crossentropy: 0.0549 - loss: 0.0549 - val_accuracy: 0.8687 - val_binary_crossentropy: 0.4277 - val_loss: 0.4277
Epoch 4/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 18s 370ms/step - accuracy: 0.9986 - binary_crossentropy: 0.0110 - loss: 0.0110 - val_accuracy: 0.8686 - val_binary_crossentropy: 0.5691 - val_loss: 0.5691
Epoch 5/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 26s 480ms/step - accuracy: 1.0000 - binary_crossentropy: 0.0015 - loss: 0.0015 - val_accuracy: 0.8698 - val_binary_crossentropy: 0.6592 - val_loss: 0.6592
Epoch 6/20
49/49 ━━━━━━━━━━━━━━━━━━

In [33]:
hist = pd.DataFrame(baseline_history.history)
hist['epoch'] = baseline_history.epoch
hist.head()

,accuracy,loss,val_accuracy,val_loss,epoch
0,1.0,0.000113,0.84552,1.672322,0
1,1.0,0.000108,0.84588,1.683549,1
2,1.0,0.000103,0.84576,1.692345,2
3,1.0,0.000098,0.84576,1.702273,3
4,1.0,0.000094,0.84556,1.711423,4


### <a name='a6'></a> 5. Porównanie wydajności modeli

In [36]:
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure()
for name, history in zip(['smaller', 'baseline', 'bigger'], [smaller_history, baseline_history, bigger_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    # najpierw loss
    for key in ['loss', 'val_loss']:
        if key in hist:
            fig.add_trace(go.Scatter(x=hist['epoch'], y=hist[key],
                                     name=f'{name}_{key}', mode='lines+markers'))

    # potem wszystkie pozostałe metryki (np. accuracy, binary_accuracy, auc, itp.)
    metrics = [k for k in hist.columns if k not in ['epoch', 'loss', 'val_loss'] and not k.startswith('val_')]
    for m in metrics:
        fig.add_trace(go.Scatter(x=hist['epoch'], y=hist[m],
                                 name=f'{name}_{m}', mode='lines+markers'))
        vm = f'val_{m}'
        if vm in hist:
            fig.add_trace(go.Scatter(x=hist['epoch'], y=hist[vm],
                                     name=f'{name}_{vm}', mode='lines+markers'))

fig.update_layout(xaxis_title='Epoki', yaxis_title='wartość', legend_title='Serie')
fig.show()


### <a name='a7'></a> 6. Metody regularyzacji

In [ ]:
from tensorflow.keras.regularizers import l2

l2_model = Sequential()
l2_model.add(Dense(16, kernel_regularizer=l2(0.001), activation='relu', input_shape=(NUM_WORDS,)))
l2_model.add(Dense(16, kernel_regularizer=l2(0.01), activation='relu'))
l2_model.add(Dense(1, activation='sigmoid'))

l2_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

l2_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 16)                160016    
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [ ]:
l2_model_history = l2_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 6s 247us/sample - loss: 0.6659 - accuracy: 0.7930 - binary_crossentropy: 0.5133 - val_loss: 0.5164 - val_accuracy: 0.8678 - val_binary_crossentropy: 0.3760
Epoch 2/20
25000/25000 [==============================] - 4s 142us/sample - loss: 0.4263 - accuracy: 0.8990 - binary_crossentropy: 0.2894 - val_loss: 0.4301 - val_accuracy: 0.8856 - val_binary_crossentropy: 0.2985
Epoch 3/20
25000/25000 [==============================] - 4s 141us/sample - loss: 0.3523 - accuracy: 0.9217 - binary_crossentropy: 0.2253 - val_loss: 0.4065 - val_accuracy: 0.8878 - val_binary_crossentropy: 0.2849
Epoch 4/20
25000/25000 [==============================] - 4s 142us/sample - loss: 0.3143 - accuracy: 0.9334 - binary_crossentropy: 0.1967 - val_loss: 0.3959 - val_accuracy: 0.8871 - val_binary_crossentropy: 0.2828
Epoch 5/20
25000/25000 [==============================] - 4s 144us/sample - loss: 0.2904 - acc

In [ ]:
fig = go.Figure()
for name, history in zip(['baseline', 'l2'], [baseline_history, l2_model_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

In [ ]:
from tensorflow.keras.layers import Dropout

dropout_model = Sequential()
dropout_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(16, activation='relu'))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(1, activation='sigmoid'))

dropout_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

dropout_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 16)                160016    
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 16)                272       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [ ]:
dropout_history = dropout_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 4s 180us/sample - loss: 0.6474 - accuracy: 0.6074 - binary_crossentropy: 0.6474 - val_loss: 0.5360 - val_accuracy: 0.8468 - val_binary_crossentropy: 0.5360
Epoch 2/20
25000/25000 [==============================] - 4s 147us/sample - loss: 0.5059 - accuracy: 0.7613 - binary_crossentropy: 0.5059 - val_loss: 0.3876 - val_accuracy: 0.8795 - val_binary_crossentropy: 0.3876
Epoch 3/20
25000/25000 [==============================] - 4s 144us/sample - loss: 0.3983 - accuracy: 0.8305 - binary_crossentropy: 0.3983 - val_loss: 0.3133 - val_accuracy: 0.8870 - val_binary_crossentropy: 0.3133
Epoch 4/20
25000/25000 [==============================] - 4s 144us/sample - loss: 0.3263 - accuracy: 0.8754 - binary_crossentropy: 0.3263 - val_loss: 0.2823 - val_accuracy: 0.8890 - val_binary_crossentropy: 0.2823
Epoch 5/20
25000/25000 [==============================] - 4s 144us/sample - loss: 0.2748 - acc

In [ ]:
fig = go.Figure()
for name, history in zip(['baseline', 'dropout'], [baseline_history, dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()